## Run

### Split system in EMT, using decoupling node

In [1]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

# Define lists for timestep and prefix
delays=(0.0001, 0.0002, 0.0004, 0.001, 0.003, 0.01)
prefixes=(delay_1e-4 delay_2e-4 delay_4e-4 delay_0.001 delay_0.003 delay_0.01)
i_intfs=(0 0 0 0 0 0)

# Iterate over the lists
for i in "${!delays[@]}"; do
  SP_RC_split_decoupled_node --timestep=0.00005 -o delay="${delays[$i]}" -o method=delay -o i-intf-0="${i_intfs[$i]}" -o prefix="${prefixes[$i]}"
done

## Validate Results

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
import matplotlib.pyplot as plt
import numpy as np

# %matplotlib widget

H_v = np.array([1e-4, 2e-4, 4e-4, 0.001, 0.003, 0.01])
H_v_legends = ["1e-4", "2e-4", "4e-4", "0.001", "0.003", "0.01"]
y_1_prev_H = [0, 0, 0, 0, 0, 0]

results_sp = rt.read_timeseries_dpsim("logs/SP_RC_monolithic/SP_RC_monolithic.csv")

results_sp_itm_delay = {}
for H_v_leg in H_v_legends:
    results_sp_itm_delay[H_v_leg] = rt.read_timeseries_dpsim(
        "logs/SP_RC_split_decoupled_delay_"
        + H_v_leg
        + "_0_0/SP_RC_split_decoupled_delay_"
        + H_v_leg
        + "_0_0.csv"
    )

pt.set_timeseries_labels(results_sp["v_2"], "v_2 - Monolithic")
pt.plot_timeseries("Co-simulation results - v_intf - delay", results_sp["v_2"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_sp["i_rline"], "i_rline - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - delay", results_sp["i_rline"])

for k, results_att_list in results_sp_itm_delay.items():
    pt.set_timeseries_labels(results_att_list["v_intf"], "v_intf - S_1, delay")
    pt.plot_timeseries(
        "Co-simulation results - v_intf - delay", results_att_list["v_intf"], "--"
    )

    pt.set_timeseries_labels(results_att_list["i_intf"], "i_intf - S_1, delay")
    pt.plot_timeseries(
        "Co-simulation results - i_intf - delay", results_att_list["i_intf"], "--"
    )

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [ ]:
time_step = 50e-6

global_error_delay = []

ts_v1_m = results_sp["v_1"]
ts_v2_m = results_sp["v_2"]
ts_i_intf_m = results_sp["i_rline"]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16, 9))

for i in range(0, len(H_v_legends)):
    H = H_v[i]

    m = int(round(H / time_step))

    ts_v1_delay = results_sp_itm_delay[H_v_legends[i]]["v_1"]
    ts_v2_delay = results_sp_itm_delay[H_v_legends[i]]["v_2_1"]
    ts_i_intf_delay = results_sp_itm_delay[H_v_legends[i]]["i_intf"]

    xy_a = np.array(
        [
            ts_v1_m.values[::m],
            ts_v2_m.values[::m],
            ts_i_intf_m.values[::m],
            ts_v2_m.values[::m],
        ]
    )
    xy_delay = np.array(
        [
            ts_v1_delay.values[::m],
            ts_v2_delay.values[::m],
            ts_i_intf_delay.values[::m],
            ts_v2_delay.values[::m],
        ]
    )

    global_error_v1_delay_t = np.abs(ts_v1_m.values[::m] - ts_v1_delay.values[::m])
    global_error_v2_delay_t = np.abs(ts_v2_m.values[::m] - ts_v2_delay.values[::m])
    global_error_i_intf_delay_t = np.abs(
        ts_i_intf_m.values[::m] - ts_i_intf_delay.values[::m]
    )

    global_error_delay_t = np.linalg.norm(xy_a - xy_delay, axis=0)

    axes[0].plot(
        ts_v1_m.time[::m],
        global_error_v1_delay_t,
        label="V1 - delay - " + H_v_legends[i],
    )
    axes[1].plot(
        ts_v2_m.time[::m],
        global_error_v2_delay_t,
        label="V2 - delay - " + H_v_legends[i],
    )

    axes[2].plot(
        ts_i_intf_m.time[::m],
        global_error_i_intf_delay_t,
        label="i_intf - delay - " + H_v_legends[i],
    )

    global_error_delay.append(np.max(np.max(global_error_delay_t)))

axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Global error")
axes[0].grid()
axes[0].legend()

axes[1].set_xlabel("t [s]")
axes[1].set_ylabel("Global error")
axes[1].grid()
axes[1].legend()

axes[2].set_xlabel("t [s]")
axes[2].set_ylabel("Global error")
axes[2].grid()
axes[2].legend()

In [ ]:
plt.figure()
plt.plot(
    np.log10(H_v),
    np.log10(np.multiply(H_v, 10 * 0.012)),
    "k",
    label=r"$\mathcal{O}(H)$",
)
plt.plot(
    np.log10(H_v),
    np.log10(np.multiply(H_v**2, 10**1.5)),
    "--k",
    label=r"$\mathcal{O}(H^2)$",
)
plt.plot(np.log10(H_v), np.log10(global_error_delay), "g", label="Delay")
plt.plot(np.log10(H_v), np.log10(global_error_delay), "go")
plt.xlabel("")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()